## Research Project 3
```text
- Source: SEC
- Goal: Extract information from HTML tables
- Techniques: HTML parsing, NER, Dataframes
- Tools: Beautifulsoup, pandas, spacy
- Lines of code: ~100```

In [1]:
# Standard library
import datetime

# Third-party
import re
import spacy
import gensim
import requests
import numpy as np
import pandas as pd
from lxml import html, etree
from bs4 import BeautifulSoup, Tag

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
class Extractor(object):
    
    def __init__(self, table, id_=None, **kwargs):
        if isinstance(table, Tag):
            self._table = table.find(id=id_)
        elif isinstance(table, bytes) or isinstance(table, str):
            self._table = BeautifulSoup(table, 'html.parser').find(id=id_)
        else:
            raise Exception('unrecognized type')
        self._output = []
        
    def parse(self):
        self._output = []
        row_ind = 0
        col_ind = 0
        for row in self._table.find_all('tr'):
            smallest_row_span = 1
            for cell in row.children:
                if cell.name in ('td', 'th'):
                    row_span = int(cell.get('rowspan')) if cell.get('rowspan') else 1
                    smallest_row_span = min(smallest_row_span, row_span)
                    col_span = int(cell.get('colspan')) if cell.get('colspan') else 1
                    while True:
                        if self._check_cell_validity(row_ind, col_ind):
                            break
                        col_ind += 1
                    self._insert(row_ind, col_ind, row_span, col_span, str(cell.get_text()))
                    col_ind += col_span
            row_ind += smallest_row_span
            col_ind = 0
        return self

    def return_list(self):
        return self._output

    def _check_validity(self, i, j, height, width):
        return all(self._check_cell_validity(ii, jj) for ii in range(i, i+height) 
                   for jj in range(j, j+width))

    def _check_cell_validity(self, i, j):
        if i >= len(self._output):
            return True
        if j >= len(self._output[i]):
            return True
        if self._output[i][j] is None:
            return True
        return False

    def _insert(self, i, j, height, width, val):
        for ii in range(i, i+height):
            for jj in range(j, j+width):
                self._insert_cell(ii, jj, val)

    def _insert_cell(self, i, j, val):
        while i >= len(self._output):
            self._output.append([])
        while j >= len(self._output[i]):
            self._output[i].append(None)
        if self._output[i][j] is None:
            self._output[i][j] = val

def levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [4]:
url = 'https://www.sec.gov/Archives/edgar/data/789019/000119312517310951/d461626ddef14a.htm'
res = requests.get(url)
tree = html.fromstring(res.content)

In [197]:
dfs = []
for table in tree.xpath('//table'):
    table_string = re.sub(r'<br/?>', '\n', str(etree.tostring(table)), flags=re.DOTALL)
    extractor = Extractor(table_string)
    extractor.parse()
    parsed = [[i.replace('\xa0', ' ').strip() for i in j] for j in extractor.return_list()]
    parsed = [i for i in parsed if any(bool(j) for j in i)]
    if parsed and parsed[0] and re.findall(r'^name', parsed[0][0], flags=re.IGNORECASE|re.DOTALL):
        parsed[0] = [' '.join(i.split()).strip() for i in parsed[0]]
        parsed[0] = [i.replace('\\n', ' ').strip() for i in parsed[0]]
        parsed[1:] = [[i.replace(r'\n', ', ').strip() for i in j] for j in parsed[1:]]
        parsed = [[i if i else None for i in j] for j in parsed]
        df = pd.DataFrame(parsed[1:], columns=parsed[0])
        if df.ix[:,0].iloc[0].replace(',', '') == df.columns[0]:
            df = df.iloc[1:]
        df = df.dropna(axis=1, how='all')
        df.columns = ['Name'] + df.columns.values.tolist()[1:]
        dfs.append(df)

In [198]:
mapping = {None: None}

for df in dfs:
    for name in df['Name'].values:
        if name is None:
            continue
        clean = re.sub(r'\d+', '', name)
        ents = [i.text for i in nlp(clean).ents if i.label_ == 'PERSON']
        if not ents: ents = [name]
        mapping[name] = ents[0]
        
for num, df in enumerate(dfs):
    df['Name'] = df['Name'].apply(lambda x: mapping[x])
    for num, row in enumerate(df.iterrows()):
        if row[1]['Name'] is None and num > 0:
            df.iloc[num]['Name'] = df.iloc[num - 1]['Name']

In [205]:
dfs[6]

,Name,Year,Salary ($),Bonus1 ($),Stock awards2 ($),Non-equity incentive plan compensation3 ($),All other compensation4 ($),Total ($)
0,Satya Nadella,2017,"1,450,000",N/A,"11,434,557","7,032,406","97,189","20,014,152"
3,Amy E. Hood,2017,"852,917",N/A,"7,015,071","3,624,896","89,260","11,582,144"
6,Jean-Philippe,2017,"751,054",N/A,"14,735,4646","2,762,884","45,214","18,294,616"
7,Margaret L. Johnson,2017,"715,000",N/A,"3,907,922","2,168,795","89,238","6,880,955"
10,Bradford L. Smith,2017,"785,833",N/A,"6,193,370","3,241,543","98,989","10,319,735"
